<a href="https://colab.research.google.com/github/drwitt/BME_590_Tensorflow_Deep_Learning/blob/master/Karpathy_RNN_on_Genesis_generating_random_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
Updated in March 2018; Last updated in Aug 2019 by wz 

@author: A.Karpathy
@editor: WZ
'''
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License


You might need to run as well: 

import nltk
nltk.download()

to get the NLTK data 


"""

'\nMinimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)\nBSD License\n\n\nYou might need also\nimport nltk\nnltk.download()\n\n\n'

Perhaps the easiest way to work with your data is to mount your google drive

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [9]:
import numpy as np

inputtxt='/content/drive/My Drive/MIDS_Yr2/IDS_690_NLP/Text_Data/english-kjv.txt'

print(inputtxt[0:500])
# data I/O

/content/drive/My Drive/MIDS_Yr2/IDS_690_NLP/Text_Data/english-kjv.txt


In [10]:
data = open(inputtxt, 'r').read() # should be simple plain text file
print(data[0:100])




In the beginning God created the heaven and the earth.
And the earth was without form, and void; and


In [11]:
chars = list(set(data))
print('chars', chars)

chars ['h', 'b', 'e', 'i', 'a', 'R', 'W', 'F', 'J', 'w', 't', 's', 'N', 'H', '?', ';', 'B', 'Z', 'U', 'u', ' ', 'f', 'k', 'T', 'M', 'q', 'c', 'P', 'n', 'y', 'd', 'S', ':', 'I', 'v', 'Y', 'K', 'x', '\n', 'g', 'r', '(', '!', 'l', ')', 'm', 'z', 'L', 'p', "'", 'E', 'C', 'G', 'j', 'o', 'A', 'D', 'O', ',', '.']


In [12]:
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))

data has 195515 characters, 60 unique.


In [13]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
print(char_to_ix)

ix_to_char = { i:ch for i,ch in enumerate(chars) }
print(ix_to_char)

{'h': 0, 'b': 1, 'e': 2, 'i': 3, 'a': 4, 'R': 5, 'W': 6, 'F': 7, 'J': 8, 'w': 9, 't': 10, 's': 11, 'N': 12, 'H': 13, '?': 14, ';': 15, 'B': 16, 'Z': 17, 'U': 18, 'u': 19, ' ': 20, 'f': 21, 'k': 22, 'T': 23, 'M': 24, 'q': 25, 'c': 26, 'P': 27, 'n': 28, 'y': 29, 'd': 30, 'S': 31, ':': 32, 'I': 33, 'v': 34, 'Y': 35, 'K': 36, 'x': 37, '\n': 38, 'g': 39, 'r': 40, '(': 41, '!': 42, 'l': 43, ')': 44, 'm': 45, 'z': 46, 'L': 47, 'p': 48, "'": 49, 'E': 50, 'C': 51, 'G': 52, 'j': 53, 'o': 54, 'A': 55, 'D': 56, 'O': 57, ',': 58, '.': 59}
{0: 'h', 1: 'b', 2: 'e', 3: 'i', 4: 'a', 5: 'R', 6: 'W', 7: 'F', 8: 'J', 9: 'w', 10: 't', 11: 's', 12: 'N', 13: 'H', 14: '?', 15: ';', 16: 'B', 17: 'Z', 18: 'U', 19: 'u', 20: ' ', 21: 'f', 22: 'k', 23: 'T', 24: 'M', 25: 'q', 26: 'c', 27: 'P', 28: 'n', 29: 'y', 30: 'd', 31: 'S', 32: ':', 33: 'I', 34: 'v', 35: 'Y', 36: 'K', 37: 'x', 38: '\n', 39: 'g', 40: 'r', 41: '(', 42: '!', 43: 'l', 44: ')', 45: 'm', 46: 'z', 47: 'L', 48: 'p', 49: "'", 50: 'E', 51: 'C', 52: 'G',

In [0]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [0]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [0]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

"""   sample a sequence of integers from the model 
h is memory state, seed_ix is seed letter for first time step """

In [0]:
def sample(h, seed_ix, n):
   x = np.zeros((vocab_size, 1))
   x[seed_ix] = 1
   ixes = []
   for t in range(n):
      h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
      y = np.dot(Why, h) + by
      p = np.exp(y) / np.sum(np.exp(y))
      ix = np.random.choice(range(vocab_size), p=p.ravel())
      x = np.zeros((vocab_size, 1))
      x[ix] = 1
      ixes.append(ix) 
   return ixes


In [0]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

### Please don't run it for 12h on colab 


In [19]:
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 10000 == 0:  #was 1000
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print( '----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 10000 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress #was 1000
  
  # perform parameter update with Adagrad
  '''wz
  Returns an iterator of tuples, where the i-th tuple contains the i-th element from each 
  of the argument sequences or iterables. 
  The iterator stops when the shortest input iterable is exhausted. 
  With a single iterable argument, it returns an iterator of 1-tuples.
  '''
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 g;SG'jW:kYh gG?K)souTHZCbyI,:(fAPB'!CfTCJZE?!C.hsTD( (c',!; NeKTqre.BChieEiLMZNdpMGKC)m;,ZIPEpK Nr.pskaqtc:oapBLHxvtsdncyfYCki:CNCcPhwZ'Hyr!MmS?wGR.FkpAg?ahlHE;SEp'AT,r:)cU
uIpifcd
wtOeNhJi.n(ambqw.cz 
----
iter 0, loss: 102.358618
----
 rdild ficald lifirkid
I hin?
And Sind, and fasefrwo LORD
him I fook shaut and upou said, and gyet kill with and, and to they sell
wfounte fim, untors I hare dryered yif thal sofs of mad Ifreve, and Ab 
----
iter 10000, loss: 44.658378
----
 olded comcatiked called, the ry
sank fith, the
roughto berous theen, thing bed, fillant thou? the sard, God sperg the
LORD shall hatt the fatll, by name him's wilen; brithd the gevered, what kard sped 
----
iter 20000, loss: 40.071115
----
 y Now the
fither with be not be was
be me mound their unto all Mimes eabret awe I heri
Jame; be thing?
Wefases years, The frund year unto were ea the wive is said
ustine it uspined the
Hableed
cime an 
----
iter 30000, loss: 38.915290
----
 f waters of eacahe the drod whara

KeyboardInterrupt: ignored

In [0]:
# prior run sampled

In [0]:
"""
----
 MlpSYgG)PscAJcCF)maYMxgcsAEMeaG'uG!awALhf
nTbnBs'?C?EmdMe''JaLW)fDFH(zHGJdbkGhZNSP:pAWYvaYycGbpo,dx(iMZWUHK
h)zLSqproOICzgvLbGo(.a, ?hK?v
kDwLoWC;PiecyfmGnE:
TLhj),gv
F.k,s;
cfL(IamKwqnNzihWusAFIROgIu 
----
iter 0, loss: 102.358623
----
 ll cworon of foed wa.
c ide wtonAD fanty faJm aol dehibe ,he thee dald y,afas. hil fofe wahan.d; th aindeethet ans mnd eiri heter
ind Nveyy
falnle vathe thallt, url, of arert spy toat and tohy ge meeo 
----
iter 1000, loss: 80.115990
----
 , I unthedering alr thar iled Gof hing thevyrovend and herth ild, yam hes
sorisu.
pet w
se
kat his how le
Ardixd I Hhile, ind
the the
mi goas bime Idind, And mald henling fomay, alaeg doter; wance I f 
----
iter 2000, loss: 65.526007
----
  in noths of chou'd Gean pavenco gid liy the the he elrs thoins min ox.
And the iod go sotoy saaven and ofhim he Noch the ahe on tee in ok thaecem, lhamthe salay.
And sava htht thricarshers
them thae  
----
iter 3000, loss: 57.927522
----
 ---
  wored all don terver,
whot liverwe vift, wuund ban anten.
And mad unto if us thy hes in thou unto her, and case shall not a thren
sputh unto mazind him Abrahah.
And they lill dounst will the king to  
----
iter 50000, loss: 42.217230
----
 
----
 
And the sais
Enou food men behold, spour him Hass in to hit fassy in theirs.
And whous Phar' shau. And Joreh en the cond goed lose, and in on youre, and it tyough.f fore, and to sast
ho, Behold be ao 
----
iter 85000, loss: 40.487233
----
  king is my said, bucass. And he soed satild youl and the bros on Poweph yeom beforest land in shalssed; as at
he notsy be Jacob I alle:
Is
ald.
And rearftour: and they tles wild ald my falleld ha fol 
----
iter 86000, loss: 39.939279
----
----
 may in
Esaughid servaget Isaach the fapleb
the begam, Unar theued and to the pare ase gad anoon bleires not sat at his sons
wast sons thee pord hiveh burost sabob him a fie said, breons
the eald shirk 
----
iter 125000, loss: 39.021318
----
 ed the mon of the earelvang, of will of God, and intt
samal.
And this hast they awale, Beho the comed douth, Pather spay madh they Leew, sseanter onding the lowtord terimese: Fearah of upon.
The
went  
----
iter 126000, loss: 38.998472
----
----
iter 168000, loss: 38.126692
----
 ee his not then, the seas tom shall said, I with silt, Os naml, when Abraht thy seevantsth, and houcher
sthat caso is Abraham's nase teme. butobhes not head came of my
are shailf begat kistrer, unto w 
----
iter 169000, loss: 38.216707
----
pots, reart, and were agired 
----
iter 199000, loss: 37.881385
----
 hen
unto the shall brouse,
fill a lothered hid with me, dake will bearath all ane wiver is forth brock that eves,
and shele upon
the graceed vawat have Rachee inger as Adat in to Jacob be agained so o 
----
iter 200000, loss: 37.349877
----
 d his pin
dreabinge out
of Jadelis, and had up his said, whit I but thele of a feet this in to
will of Hasron.
And ithedd I dweldy mured's wi livedter he said, Whouseart shall Bechel
in the in ano, an 
----
----
iter 218000, loss: 37.791892
----
 hy agoin of the forchemebes Esaurn tham neraned hiigst years
his owered at shall the brepet perdinge him
Esiin remivering, wen, wheirn and the lersin unto the eard
they up to thy giver tole offerd off 
----
iter 219000, loss: 37.282680
----
 's of the king of hie they are
not. Sorn me intt of thy cpae undremo.
And the
youprnas, bort unto
the satelerp, and in that the ray, Learth.
And the LORDarresh of hered is went LORD dreing madefe thou 
----
iter 220000, loss: 37.477838
----
 ilt said there
of wad Abramaen dothy will lad ix the gilt with begai go dimied chanted, and give him upon the meish bach of thounstn his with my may ofly wifl it this sent, as no benceay.
And they, an 

"""